In [1]:
!wget http://www.eraserbenchmark.com/zipped/multirc.tar.gz
!tar -xvzf multirc.tar.gz
!mv multirc/val.jsonl multirc/dev.jsonl

--2020-05-13 12:58:42--  http://www.eraserbenchmark.com/zipped/multirc.tar.gz
Resolving www.eraserbenchmark.com (www.eraserbenchmark.com)... 3.224.101.150, 34.204.135.175, 52.71.209.190, ...
Connecting to www.eraserbenchmark.com (www.eraserbenchmark.com)|3.224.101.150|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1667550 (1.6M) [application/gzip]
Saving to: ‘multirc.tar.gz’

multirc.tar.gz      100%[===================>]   1.59M  --.-KB/s    in 0.08s   

2020-05-13 12:58:42 (19.0 MB/s) - ‘multirc.tar.gz’ saved [1667550/1667550]

multirc/
multirc/docs/
multirc/docs/News_CNN_cnn-3b31c717af0ec349ef0a067373d6444e1a05c414.txt
multirc/docs/Society_Law_and_Justice_masc-A_defense_of_Michael_Moore-10.txt
multirc/docs/Science-textbook_science-g3-10.txt
multirc/docs/wikiMovieSummaries_3965765.txt
multirc/docs/Science-textbook_science-g4-47.txt
multirc/docs/Sept11-reports_oanc-chapter-6-22.txt
multirc/docs/Sept11-reports_oanc-chapter-1-6.txt
multirc/docs/Fiction-stories_

In [5]:
import json
from Datasets.utils import load_flattened_documents
documents = load_flattened_documents('multirc', docids=None)
def read_annotations(json_file) :
    anns = [json.loads(line) for line in open(json_file)]
    for a in anns :
        doc_id = a['annotation_id'].split(':')
        assert len(doc_id) == 3
        doc_id = doc_id[0]
        a['document'] = " ".join(documents[doc_id])
        a['label'] = a['classification']
        del a['classification']
        del a['docids']
        a['rationale'] = []
        for evgroup in a['evidences'] :
            for ev in evgroup :
                assert ev['docid'] == doc_id
                a['rationale'].append((ev['start_token'], ev['end_token']))
        del a['evidences']
        del a['query_type']
        
    return anns

In [6]:
import os
os.makedirs('data/', exist_ok=True)

In [7]:
for key in ['train', 'dev', 'test'] :
    ann = read_annotations('multirc/' + key + '.jsonl')
    with open('data/' + key + '.jsonl', 'w') as f :
        f.write('\n'.join([json.dumps(line) for line in ann]))